## PyQE Patient Count Query Demo

1) Setup the imports

In [1]:
# Path has to be set before importing pyqe
import sys, os
sys.path.append(os.path.join(sys.path[0],'..', '..'))

import json
from pyqe import *

2) Define authenticated query with name

In [2]:
query = Query('Number_of_female_patients_born_in_1930s_not_from_California_or_Florida')

10-Jan-2024 15:55:06 - INFO - Username is: admin
10-Jan-2024 15:55:06 - INFO - Authenticating with CFUAA..
10-Jan-2024 15:55:08 - INFO - Authentication successful


3) Define patient filter

In [3]:
query.set_study('703c5d8a-a1d9-4d42-a314-5b9aad513390')

In [4]:
female_patient = Person.Patient()
female_constraint = Constraint().add(Expression(ComparisonOperator.EQUAL, 'Female'))
female_patient.add_gender([female_constraint])

not_ca_state_constraint = Constraint().add(Expression(ComparisonOperator.NOT_EQUAL, 'CA'))
not_fl_state_constraint = Constraint().add(Expression(ComparisonOperator.NOT_EQUAL, 'FL'))
female_patient.add_state([not_ca_state_constraint, not_fl_state_constraint])

year_of_birth_constraint = Constraint()
year_of_birth_constraint.add(Expression(ComparisonOperator.MORE_THAN_EQUAL, '1930'))
year_of_birth_constraint.add(Expression(ComparisonOperator.LESS_THAN_EQUAL, '1939'))
female_patient.add_year_of_birth([year_of_birth_constraint])

4) Generate patient count filter request by adding the defined filter

In [5]:
query.add_filters([female_patient])
request = query.get_patient_count_filter()

5) Get patient count with request

In [6]:
patient_count = Result().get_patient_count(request)
print(f'\nTotal female patients born in 1930s not from California or Florida: {patient_count}')
cohort = query.get_dataframe_cohort()
filter_cohortId = 39
patient_countby_cohort_id = Result().get_patient_count_by_cohortId(cohort, filter_cohortId)
print(f'\nTotal female patients born in 1930s not from California or Florida for cohort id {filter_cohortId}: {patient_countby_cohort_id}')


Total female patients born in 1930s not from California or Florida: 211

Total female patients born in 1930s not from California or Florida for cohort id 39: 52


6) Add condition occurrence for low back pain or type 2 diabetes, generate request & get patient count

In [ ]:
condition = Interactions.ConditionOccurrence('')
low_back_pain_constraint = Constraint().add(Expression(ComparisonOperator.EQUAL, 'Low back pain'))
type_2_diabetes_constraint = Constraint().add(Expression(ComparisonOperator.EQUAL, 'Type 2 diabetes mellitus'))
condition.add_conditionname([low_back_pain_constraint, type_2_diabetes_constraint])
query.add_filters([condition])

request = query.get_patient_count_filter()
patient_count = Result().get_patient_count(request)
print(f'\nTotal female patients born in 1930s not from California or Florida with Low back pain or Type 2 diabetes mellitus: {patient_count}')


Total female patients born in 1930s not from California or Florida with Low back pain or Type 2 diabetes mellitus: 165


7) Define authenticated concept code query with name

In [7]:
concept_query = Query('Number_of_male_patients_with_outpatient_visits')
concept_query.set_study('703c5d8a-a1d9-4d42-a314-5b9aad513390')

8) Define criteria groups

In [8]:
male_patient = Person.Patient()
male_patient.add_concept_set(ConceptSet('Male', Domain.GENDER, ['M']))

outpatient_visit = Interactions.Visit('Outpatient Visit')
outpatient_visit.add_concept_set(ConceptSet('Outpatient Visit', Domain.VISIT, ['OP']))

9) Generate request & get patient count

In [10]:
male_outpatient_visit_group = CriteriaGroup(MatchCriteria.ALL,
                                            [male_patient, outpatient_visit])
concept_query.add_criteria_group(male_outpatient_visit_group)
concept_request = concept_query.get_patient_count_filter()

concept_result = Result()
concept_patient_count = concept_result.get_patient_count(concept_request)
print(f'\nTotal male patient with outpatient visits: {concept_patient_count}')

Domain.GENDER
Domain.VISIT

Total male patient with outpatient visits: 0


10) Create a phenotype example (sleep apnea)

In [ ]:
conceptQuery = ConceptQuery()
sleep_apnea_dx_concept_codes = ['313459', '41975002', '79280005', '78275009']
sleep_apnea_dx_concept_ids = [conceptQuery.get_standard_concept_ids(concept_code)[0]['conceptId'] for concept_code in sleep_apnea_dx_concept_codes]

sleep_apnea_device_concept_codes = ['A7034', 'A9270', 'E0470', 'E0471', 'E0601']
sleep_apnea_device_concept_ids = [conceptQuery.get_standard_concept_ids(concept_code)[0]['conceptId'] for concept_code in sleep_apnea_device_concept_codes]

sleep_apnea_snomed_procedure_concept_code = '60554003'
sleep_apnea_snomed_procedure_concept_id = conceptQuery.get_standard_concept_ids(sleep_apnea_snomed_procedure_concept_code, 'SNOMED')[0]['conceptId']

In [ ]:
phenotype_query = Query('Total_Patients_With_Sleep_Apnea')
phenotype_query.set_study('703c5d8a-a1d9-4d42-a314-5b9aad513390')
sleep_apnea_dx = ConceptSet('Sleep Apnea Dx', Domain.CONDITION, sleep_apnea_dx_concept_ids)
sleep_apnea_dx_condition = Interactions.ConditionOccurrence(sleep_apnea_dx.name)
sleep_apnea_dx_condition.add_concept_set(sleep_apnea_dx)

sleep_apnea_device = ConceptSet('Sleep Apnea Device', Domain.DEVICE, sleep_apnea_device_concept_ids)
sleep_apnea_device_exposure = Interactions.DeviceExposure(sleep_apnea_device.name)
sleep_apnea_device_exposure.add_concept_set(sleep_apnea_device)


phenotype_query.add_criteria_group(CriteriaGroup(MatchCriteria.ALL, [sleep_apnea_dx_condition, sleep_apnea_device_exposure]))
phenotype_request = phenotype_query.get_patient_count_filter()

phenotype_patient_count = Result().get_patient_count(phenotype_request)
print(
    f'\nTotal patients with Sleep Apnea condition and device exposure: {phenotype_patient_count}')

Domain.CONDITION
Domain.DEVICE

Total patients with Sleep Apnea condition and device exposure: 16
